## 1.	MNIST: ejemplo de clasificación de imágenes

### 1.2. MNIST dataset 

* El MNIST (Modified National Institute of Standards and Technology) es una base de datos de dígitos manuscritos.
* Se utiliza habitualmente para entrenar y probar sistemas de procesamiento de imágenes en el campo del aprendizaje automático. Se podría decir que es el *Hello World* de este campo.
* Se compone de 60.000 imágenes de entrenamiento y 10.000 imágenes de prueba.
* Cada imagen tiene unas dimensiones de 28x28 píxeles en escala de grises y representa uno de los diez dígitos posibles (del 0 al 9).
  
<!--- * El conjunto de entrenamiento será un tensor 3D de `(60000, 28, 28)` y el conjunto de test un tensor 3D de `(10000, 28, 28)` siendo el `dtype` un `uint8`. --->

<img src="images/classical_nn_implementation/mnist.png" width=594 height=361/>

*  Los datos MNIST se precargan en Keras en forma de cuatro arrays NumPy: dos pares para entrenamiento y dos pares para test
* Cada par (entrenamiento o test) tiene un array para almacenar los datos de la imagen y un array con las etiquetas correspondientes a cada imagen.
* Importamos el paquete `mnist` que almacena los datos correspondientes.

In [5]:
from tensorflow.keras.datasets import mnist

* Cargamos los datos de entrenamiento y los datos de prueba como un objeto NumPy `ndarray`.
* NumPy (http://www.numpy.org/) es el paquete fundamental para la computación científica con Python.
*  Contiene, entre otras cosas:
 - Un objeto para representar arrays n-dimensionales (`ndarray`) eficientemente.
    - Operaciones aritméticas rápidas orientadas a arrays con capacidades de difusión flexibles.
    - Funciones matemáticas que permiten realizar eficientemente operaciones sobre arrays sin tener que escribir bucles.

In [32]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [33]:
type(train_images)

numpy.ndarray

* Veamos los datos de entrenamiento.
* `shape` es un atributo de `ndarray`.
* Contiene una tupla de enteros indicando el tamaño de cada una de las dimensiones del array. 
* La longitud de la tupla será el número de dimensiones de la matriz.

In [34]:
train_images.shape

(60000, 28, 28)

In [35]:
train_labels.shape

(60000,)

In [36]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

* Los datos para el test:

In [37]:
test_images.shape

(10000, 28, 28)

In [38]:
test_labels.shape

(10000,)

In [39]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

### 1.3. Preparando los datos 

* Necesitamos preparar los datos a lo que la red espera de ellos.
* Las imágenes son un array de dimensiones `(60000, 28, 28)` de tipo `uint8` con valores de `[0, 255]`.
* En  primer lugar, deben ser transformadas en una matriz de dimensiones `(60000, 28 * 28)` 
 - La imagen se aplana a una simple matriz de `28 * 28 = 784` valores.
  - Usamos la función NumPy `reshape` que cambia la forma de un array sin cambiar sus datos (https://numpy.org/doc/stable/reference/generated/numpy.reshape.html)

In [40]:
train_images = train_images.reshape((60000, 28 * 28))
test_images = test_images.reshape((10000, 28 * 28))

* En segundo lugar, deben ser transformados a un tipo float `float32` con valores entre 0 y 1.
    - Cada valor va de 0 a 255 (niveles de gris) y debe ser normalizado a valores entre 0 y 1 (dividiendo por 255).
    - Usamos la función NumPy `astype` que copia el array y cast sus valores a un tipo especificado.
    - Cuando dividimos un `ndarray` por un número dividimos cada elemento del array por ese número.

In [41]:
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

* También necesitamos codificar categóricamente las etiquetas utilizando la codificación *one-hot*.
* En esta codificación convertimos un entero en una matriz de enteros en la que todos los valores se ponen a cero excepto el valor que correspondía al entero original que se pone a uno.
* Usamos la función `to_categorical` que convierte enteros en una matriz binaria de clases (https://keras.io/api/utils/python_utils/)

In [42]:
train_labels[2]

4

In [43]:
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [44]:
train_labels[2]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

### 1.4. Building the Model and Layers

#### Modelo secuencial


* Utilizaremos el modelo secuencial de Keras.
* Usaremos la clase `Sequential` (https://keras.io/api/models/sequential/) para crear el modelo secuencial.

In [45]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()

#### Capa de entrada
* La primera capa que se debe crear es una capa de entrada que recibe la entrada (https://keras.io/api/layers/core_layers/input/).
* El parámetro más importante en el constructor de `Input` es `shape`, una tupla de enteros que indica la forma de la entrada.
* Por ejemplo, `shape=(784,)` indica que la entrada esperada serán lotes de vectores de 784 dimensiones (nuestras imágenes de 28x28).
* Los elementos de esta tupla pueden ser `None`, representando dimensiones donde la forma no es conocida.
* Añadimos este capa al modelo usando el método `add`.

In [46]:
model.add(layers.Input(shape=(784, )))

#### Capas densas

* Las capas densas son capas formadas por neuronas artificiales, cada una está completamente conectada con las capas anteriores y posteriores.
* En Keras, `Dense` (https://keras.io/api/layers/core_layers/dense/) representa una capa de red neuronal densamente conectada que implementa la operación `output = activation(dot(input, kernel) + bias)`.
* Los parámetros típicos para el constructor `Dense` son:
    - units: el número de neuronas en la capa.
    - activation: la función de activación utilizada en cada neurona.
    - input_shape: cuando se pasa, Keras creará una capa de entrada para insertar antes de la capa actual. Esto puede tratarse como equivalente a definir explícitamente una capa `Input`.
* En este caso, vamos a crear una capa densa que recibe 784 entradas y produce 512 salidas activadas por la función ReLU.

In [47]:
model.add(layers.Dense(512, activation="relu"))

#### Capa de salida

* La capa de salida también es una capa densa, pero en este caso con 10 neuronas, cada una de las cuales representará un dígito.
* El valor de salida de cada neurona de la capa de salida indicará la probabilidad de que el dígito actual corresponda al dígito representado por esa neurona.
* La probabilidad total debe sumar uno.
* Para obtener estos valores, utilizamos la función de activación *softmax*.

In [48]:
model.add(layers.Dense(10, activation="softmax"))

#### Función de activación softmax

* La función softmax o función exponencial normalizada convierte un vector de $K$ números reales en una distribución de probabilidad de $K$ resultados posibles.

* La función softmax se utiliza a menudo como la última función de activación de una red neuronal para normalizar la salida de una red a una distribución de probabilidad sobre las clases de salida predichas.

* La función softmax estándar $\sigma : \mathbb{R}^K \to (0, 1)^K$ se define cuando $K \ge 1$ mediante la fórmula:

\begin{equation*}
\sigma(\mathbf{z})_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}} \ \ \text{ para } i = 1, \dotsc, K \text{ y } \mathbf{z} = (z_1, \dotsc, z_K) \in \mathbb{R}^K.
\end{equation*}

<img src="images/classical_nn_implementation/softmax.png" width=800 />

#### Resumen del modelo

* Podemos observar un resumen de nuestro modelo utilizando la función `summary()`.

In [49]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

### 1.5. Proceso de aprendizaje

#### Definiendo los parámetros de aprendizaje

* Antes de entrenar un modelo, es necesario configurar el proceso de aprendizaje, lo cual se hace a través del método `compile`.
* Necesitamos elegir tres cosas más como parte del paso de compilación:
    - Una **función de pérdida**: Cómo el modelo podrá medir su rendimiento en los datos de entrenamiento y, por lo tanto, cómo podrá orientarse en la dirección correcta.
    - Un **optimizador**: El mecanismo a través del cual el modelo se actualizará en función de los datos de entrenamiento que vea, para mejorar su rendimiento.
    - **Métricas** para monitorear durante el entrenamiento y la prueba. Aquí, solo nos importará la precisión (la fracción de las imágenes que fueron clasificadas correctamente).

In [50]:
model.compile(optimizer="rmsprop", 
              loss="categorical_crossentropy",
              metrics=["accuracy"])

#### Entrenamiento

* Ahora estamos listos para entrenar el modelo, lo cual en Keras se hace a través de una llamada al método `fit()` del modelo; ajustamos el modelo a sus datos de entrenamiento.

* Los parámetros principales del método `fit()` son (https://keras.io/api/models/model_training_apis/):
    - Los **datos de entrada**: las imágenes de entrenamiento.
    - Los **datos objetivo**: las etiquetas de entrenamiento.
    - Las **épocas**: Número de iteraciones sobre todos los datos de entrada y objetivo. 
        * Al usar los datos de entrada varias veces, aumentamos la precisión de nuestro modelo. 
        * Pero debemos tener cuidado de no causar sobreajuste.
    - El **tamaño del lote**: Número de muestras por actualización de gradiente.
        * Con un tamaño de lote de 1, actualizamos los pesos de la red después de que cada muestra ha pasado a través de la red, esto es lento y consume muchos recursos.
        * Podemos agrupar los datos de entrada en lotes y solo actualizar los pesos después de que todas las muestras del lote hayan ingresado a la red.
        * El aprendizaje consume menos recursos, pero dado que estamos propagando hacia atrás el error promedio de todas las muestras del lote, la calidad del modelo puede degradarse y, en última instancia, puede no ser capaz de generalizar bien en datos que no ha visto antes.

* Se muestran dos cantidades durante el entrenamiento: la pérdida del modelo sobre los datos de entrenamiento y la precisión del modelo sobre los datos de entrenamiento.

In [51]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8698 - loss: 0.4483
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9671 - loss: 0.1117
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9783 - loss: 0.0745
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9848 - loss: 0.0512
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9891 - loss: 0.0367


In [52]:
loss_and_metrics = model.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9771 - loss: 0.0756


# 2. Práctica a realizar

* Ahora, trataremos de implementar un modelo de red neuronal para el dataset **Fashion-MNIST**:

    - Fashion-MNIST es un conjunto de imágenes sobre diferentes artículos. Consta de un conjunto de entrenamiento de 60,000 ejemplos y un conjunto de prueba de 10,000 ejemplos.
     
    - Cada ejemplo es una imagen en escala de grises de 28x28, asociada con una etiqueta de las siguientes 10 clases: (0) Camiseta/top, (1) Pantalón, (2) Suéter, (3) Vestido, (4) Abrigo, (5) Sandalia, (6) Camisa, (7) Zapatilla, (8) Bolsa y (9) Botín.



<img src="images/classical_nn_implementation/dataset-cover.png" width=800 />


* El conjunto de datos se puede descargar fácilmente desde Keras utilizando las siguientes instrucciones:

In [8]:
from tensorflow.keras.datasets import fashion_mnist

# Cargar el dataset de Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


* A partir de aquí, debereis de completar los siguientes apartados:

1. Preprocesamiento.
    - Preprocesar el conjunto de datos para prepararlo para alimentar la red neuronal: aplanar las imágenes, convertir enteros a valores flotantes, codificar las etiquetas utilizando la codificación one-hot, etc.
    - Dividir los datos de entrenamiento en entrenamiento y validación, utilizando este último como referencia para la afinación de hiperparámetros.
<br></br>
2. Desarrollo del modelo.
    - Decidir los hiperparámetros estructurales de la red: capas, neuronas por capa, funciones de activación, etc.
    - Decidir los hiperparámetros de aprendizaje de la red: optimizador, tasa de aprendizaje, épocas, tamaño de lote, métricas, etc.
    - Justificar las decisiones tomadas.
<br></br>

3. Resultados.
    - Ejecutar y comentar los resultados obtenidos en cada paso.
    - Decidir qué métrica sería más útil para el problema en cuestión.
    - Hablar sobre posibles acciones a tomar para mejorar los valores: Técnicas de regularización, aumento/reducción de los conjuntos usados... 
